In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
import requests
import pymongo
from flask import Flask
#flask_pymongo import is generating an error code. Pip install Flask-Pymongo
##documentation here: https://flask-pymongo.readthedocs.io/en/latest/
from flask_pymongo import PyMongo
from flask import render_template, redirect
###pip install fixed the error.

In [4]:
#Set up the executable path and the browser. Boiler plate code.
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False) 

In [5]:
#link to the url from the assignment readme
redplanet_url = 'https://redplanetscience.com/'
browser.visit(redplanet_url)

In [16]:
#create the html, soup, and do the scraping
html = browser.html

redplanet_soup = bs(html, 'html.parser')

#assign as variables the scraped data
news_title = redplanet_soup.find_all('div', class_='content_title')[0].text
news_paragraph = redplanet_soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print("=================================================")
print(news_paragraph)

Space History Is Made in This NASA Robot Factory
From rockets to rovers, JPL's Spacecraft Assembly Facility has been at the center of robotic spaceflight. Here's a closer look at what makes it so special.


In [22]:
# JPL Mars Space Images - Featured Image
##Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).
###Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
####Make sure to find the image url to the full size `.jpg` image.
#####Make sure to save a complete url string for this image.

jpl_image_url = 'https://spaceimages-mars.com'
image_url = 'https://spaceimages-mars.com/image/?search=&category=Mars'

browser.visit(image_url)

browser.find_by_id('featured').click()

html = browser.html

image_soup = bs(html, 'html.parser')

image_path = image_soup.find_all('img')["src"]
featured_image_url = jpl_image_url + image_path
print(featured_image_url)


ElementDoesNotExist: no elements could be found with id "featured"